In [105]:
def infoGain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N))-N/(P+N)*math.log2(N/(P+N))

In [106]:
def insertNode(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertNode(v, d, Node)
    if d in tree:
        if isinstance(tree[d], dict):
            tree[d][Node] = 'None'
        else:
            tree[d] = {Node:'None'} 
    return tree

In [107]:
def insertConcept(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertConcept(v, d, Node)
    if d in tree:
        tree[d] = Node
    return tree

In [108]:
def getNextData(data,Node,d):
    iData = (data[Node] == d)
    nD = data[iData]
    nD = nD.drop(Node,axis=1)
    return nD

In [115]:
def getNextNode(data,AttributeList,concept,conceptVals,tree,d):
    Total=data.shape[0]
    if Total == 0: #If no Attibute then return tree
        return tree
    
    countC={}
    for cVal in conceptVals: #If Example is positive then return positive and If negative then return negative
        cData = (data[concept] == cVal)
        dataCC = data[cData]
        countC[cVal] = dataCC.shape[0]

        if countC[cVal] == 0 and cVal == 'No':
            tree = insertConcept(tree,d,'Yes')
            return tree
        if countC[cVal] == 0 and cVal == 'Yes':
            tree = insertConcept(tree,d,'No')
            return tree
    
    ClassEntropy = infoGain(countC['Yes'],countC['No']) #Class Entropy for d
    
    Attr = {}#Attribute dict holding list of possible values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
        
    AttrCount = {} #Get the attribute values being positive and negative
    TotalInfo = {}
    InfoGain = {}
    EntropyAttr = {}
    for att in AttributeList:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = (data[att] == vals)
                cData = (data[concept] == c)
                dataAtt = data[iData][cData]
                AttrCount[c] = dataAtt.shape[0]
            TotalInfo[vals] = AttrCount['Yes'] + AttrCount['No']
            if AttrCount['Yes'] == 0 or AttrCount['No'] == 0:
                InfoGain[vals]=0
            else:
                InfoGain[vals] = infoGain(AttrCount['Yes'],AttrCount['No'])
            
            if att not in EntropyAttr:
                EntropyAttr[att] = ( TotalInfo[vals] / Total ) * InfoGain[vals]
            else:
                EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[vals] / Total ) * InfoGain[vals]
    
    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g]
#     print("Gain : ",Gain)
    Node = max(Gain,key=Gain.get)
#     print("Next root : ",Node)
    tree = insertNode(tree,d,Node)
    for nD in Attr[Node]:
        tree = insertNode(tree,Node,nD)
        newData = getNextData(data,Node,nD)
        AttributeList=list(newData)[:-1]
        tree = getNextNode(newData,AttributeList,concept,conceptVals,tree,nD)
    return tree

In [116]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('play.csv')
    AttributeList=list(data)[:-1]
    concept=str(list(data)[-1])
    conceptVals = list(set(data[concept]))
    tree = getNextNode(data,AttributeList,concept,conceptVals,{'root':'None'},'root')
    print(tree)

In [117]:
main()

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


{'root': {'Outlook': {'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Overcast': 'Yes', 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}}
